In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
np.random.seed(42)

# parameters
m_train = 1000
m_test = 5000

mu_plus = np.array([0., 2.])
mu_minus = np.array([0., -2.])
Sigma = np.array([[1.5, 0.0],
                  [0.0, 1.5]])

n_train_per_class = m_train // 2
n_test_per_class = m_test // 2

# training sample
X_train_pos = np.random.multivariate_normal(mu_plus, Sigma, size=n_train_per_class)
X_train_neg = np.random.multivariate_normal(mu_minus, Sigma, size=n_train_per_class)

y_train_pos = np.ones(n_train_per_class, dtype=int)      # +1
y_train_neg = -np.ones(n_train_per_class, dtype=int)     # -1

X_train = np.vstack([X_train_pos, X_train_neg])
y_train = np.concatenate([y_train_pos, y_train_neg])

# test sample
X_test_pos = np.random.multivariate_normal(mu_plus, Sigma, size=n_test_per_class)
X_test_neg = np.random.multivariate_normal(mu_minus, Sigma, size=n_test_per_class)

y_test_pos = np.ones(n_test_per_class, dtype=int)
y_test_neg = -np.ones(n_test_per_class, dtype=int)

X_test = np.vstack([X_test_pos, X_test_neg])
y_test = np.concatenate([y_test_pos, y_test_neg])

# samples' permutation
perm_train = np.random.permutation(m_train)
X_train = X_train[perm_train]
y_train = y_train[perm_train]

perm_test = np.random.permutation(m_test)
X_test = X_test[perm_test]
y_test = y_test[perm_test]

X_train.shape, X_test.shape


((1000, 2), (5000, 2))